# Energy Dissagregation #

Estima el conusmo individual de los equipos a partir del consumo agregado. Se debe usar formato REDD

In [3]:
import warnings
warnings.filterwarnings("ignore")

import nilmtk
from nilmtk import *
from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
from nilmtk.utils import print_dict
from nilmtk.disaggregate import CombinatorialOptimisation, fhmm_exact

from pylab import rcParams
import matplotlib.pyplot as plt
%matplotlib inline
rcParams['figure.figsize'] = (14, 6)
plt.style.use('ggplot')

import numpy as np
import pandas as pd
from os.path import join

In [5]:
redd = DataSet('/home/mpoloni/Escritorio/Heartz/datasets/agraciada/',format='CSV')

In [7]:
print(redd.__doc__)


    Attributes
    ----------
    buildings : OrderedDict
        Each key is an integer, starting from 1.
        Each value is a nilmtk.Building object.

    store : nilmtk.DataStore

    metadata : dict
        Metadata describing the dataset name, authors etc.
        (Metadata about specific buildings, meters, appliances etc.
        is stored elsewhere.)
        See nilm-metadata.readthedocs.org/en/latest/dataset_metadata.html#dataset
    


In [10]:
print_dict(redd.metadata)

In [11]:
casa = redd.buildings[1]
elec = casa.elec
print(elec)

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='REDD', site_meter, appliances=[])
)


In [28]:
print(elec.__doc__)

A group of ElecMeter objects. Can contain nested MeterGroup objects.

    Implements many of the same methods as ElecMeter.

    Attributes
    ----------
    meters : list of ElecMeters or nested MeterGroups
    disabled_meters : list of ElecMeters or nested MeterGroups
    name : only set by functions like 'groupby' and 'select_top_k'
    


In [31]:
print(elec.__doc__)

A group of ElecMeter objects. Can contain nested MeterGroup objects.

    Implements many of the same methods as ElecMeter.

    Attributes
    ----------
    meters : list of ElecMeters or nested MeterGroups
    disabled_meters : list of ElecMeters or nested MeterGroups
    name : only set by functions like 'groupby' and 'select_top_k'
    


In [12]:
elec.appliances

[]

In [14]:
print(elec.mains())

ElecMeter(instance=1, building=1, dataset='REDD', site_meter, appliances=[])


In [23]:
print(elec.mains().__doc__)

Represents a physical electricity meter.

    Attributes
    ----------
    appliances : list of Appliance objects connected immediately downstream
      of this meter.  Will be [] if no appliances are connected directly
      to this meter.

    store : nilmtk.DataStore

    key : string
        key into nilmtk.DataStore to access data.

    metadata : dict.
        See http://nilm-metadata.readthedocs.org/en/latest/dataset_metadata.html#elecmeter

    STATIC ATTRIBUTES
    -----------------

    meter_devices : dict, static class attribute
        See http://nilm-metadata.readthedocs.org/en/latest/dataset_metadata.html#meterdevice
    


In [25]:
print_dict(elec.mains().metadata)

In [27]:
print_dict(elec.mains().meter_devices)

In [32]:
print(elec.mains().total_energy())

KeyError: "[('power', 'active')] not in index"

In [20]:
co = CombinatorialOptimisation()
co_train = co.train(elec)

Done training!


In [22]:
disag_filename = join('/home/matias/Descargas/Heartz/datasets/agraciada/', 'disag-co.h5')
output = HDFDataStore(disag_filename, 'w')
co.disaggregate(elec.mains(), output)
output.close()

RuntimeError: The model needs to be instantiated before calling `disaggregate`.  For example, the model can be instantiated by running `train`.